<a href="https://colab.research.google.com/github/g-twombly/FSRI-Research/blob/master/batmanTOI1130.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install batman-package

  Created wheel for batman-package: filename=batman_package-2.4.6-cp36-cp36m-linux_x86_64.whl size=212721 sha256=41db2a2d628d62f75a79f5ce870e16cb724af2683907e6079fe16c5e4c6804be
  Stored in directory: /root/.cache/pip/wheels/f6/1e/90/3b68a7004b60a358389fe48c9f9f9c0d7293e01ea562f7116f
Successfully built batman-package


In [ ]:
import batman
import numpy as np
import matplotlib as plt
%matplotlib inline

#likelihood function
def log_likelihood(theta, x, y, yerr):
  t0_2, rp, ecc_2, log_f = theta
  params2 = batman.TransitParams()
  params2.t0 = t0_2                   #from theta
  params2.per = 4.066499              #from paper
  params2.rp = rp                     #from theta
  params2.a = 0.04394                 #from paper
  params2.inc = 87.98                 #from paper
  params2.ecc = ecc_2                 #from theta
  params2.w = 0                       #(set this to 0)
  params2.limb_dark = "nonlinear"
  params2.u = [0.5,0.1,0.1,0.1] #(set these to be constant) 
  
  t2 = np.linspace(-0.025, 0.025, 100)
  m = batman.TransitModel(params2, t2)    #initializes model
  sigma2 = (yerr ** 2 + m.light_curve(params2)) ** 2 * np.exp(2 * log_f)
  return -0.5 * np.sum((y - m.light_curve(params2)) ** 2 / sigma2 + np.log(sigma2))

In [ ]:
from scipy.optimize import minimize

nll = lambda *args: -log_likelihood(*args)
#initial = np.array([t0, per, a, inc, ecc, np.log(f_true)]) + 0.1 * np.random.randn(3)
initial = np.array(log_likelihood([2458658.74627,3.65,0.22,np.log(f)], )) + 0.1 * np.random.randn(3)
soln = minimize(nll, initial, args=(x, y, yerr))
per_ml, a_ml, inc_ml, w_ml, log_f_ml = soln.x

flux = m.light_curve(params2) 
  
radii = np.linspace(0.09, 0.11, 20)
for r in radii:
      params2.rp = r                           #updates planet radius
      new_flux = m.light_curve(params2)        #recalculates light curve

ld_options = ["uniform", "linear", "quadratic", "nonlinear"]
ld_coefficients = [[], [0.3], [0.1, 0.3], [0.5, 0.1, 0.1, -0.1]]

plt.figure()

for i in range(4):
      params2.limb_dark = ld_options[i]          #specifies the LD profile
      params2.u = ld_coefficients[i]             #updates LD coefficients
      m = batman.TransitModel(params2, t)        #initializes the model
      flux = m.light_curve(params2)              #calculates light curve
      plt.plot(t, flux, label = ld_options[i])

# plt.errorbar(x, y, yerr=yerr, fmt=".k", capsize=0)
# plt.plot(m.light_, "k", alpha=0.3, lw=3, label="Not Fitted to Data")
# plt.plot(, "r", alpha=0.5, lw=4, label="Fitted Model to Data")
# plt.legend(fontsize=14)
# plt.xlim(0, 10)
# plt.xlabel("x")
# plt.ylabel("y");

NameError: ignored